# Alice

* Alice wants to send a message to Bob.
* She can communicate with Bob over an unsecure, classical channel.
* They can also communicate over a quantum channel.

In [ ]:
# Raspberry Pi exclusive imports:
from relay_lib_seeed import *  # to control the laser relay.

Alice establishes a classical channel to Bob. Anyone can listen in:

In [ ]:
# server = "broker.emqx.io"  # fallback server.
mqtt_broker_ip = "broker.emqx.io"

channel = "quantum_cryptography/classical_channel/alice/laser"

import paho.mqtt.client as mqtt

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))


def on_publish(client, userdata, result):
    print("message published")


client = mqtt.Client()
client.on_connect = on_connect
client.on_publish = on_publish

client.connect(mqtt_broker_ip, 1883, 60)

**Step 2**: Alice sends a message to Bob:

In [ ]:
plaintext = input()
_ = client.publish(channel, payload=plaintext, qos=0, retain=False)

# setup

* read input from user
* calculate number of required bits to exchange
* send it to Bob

In [ ]:
import paho.mqtt.client as mqtt
import json

mqtt_broker_ip = "broker.emqx.io"
channel = "quantum_cryptography/classical_channel"

client = mqtt.Client()
client.connect(mqtt_broker_ip, 1883, 60)

message = input()

key_length = len(message.encode("utf-8")) * 8
safety_factor = 2

setup_data = {"bits_to_exchange": key_length * safety_factor}
payload = json.dumps(setup_data)

client.publish(channel, payload=payload)

## create list of random polarizations

In [ ]:
from enum import Enum
from random import choice


class Basis(Enum):
    # 0° | or _ 90°:
    HV = 0
    # -45° \ or 45° /:
    LR = 1


polarizations = list()

for i in range(setup_data["bits_to_exchange"]):
    polarizations.append(choice(list(Basis)))

for i, b in enumerate(polarizations):
    print(i, b, sep="\t")

# quantum channel

In [ ]:
from time import sleep
from qcrypt.hardware.filterwheel import Filterwheel, Orientation
from qcrypt.hardware.laser import Laser

broker = "broker.emqx.io"

filterwheel = Filterwheel(
    username="Alice", mqtt_broker_ip=broker, dir_pin=17, step_pin=18
)
laser = Laser(username="Alice", mqtt_broker_ip=broker)

In [ ]:
filterwheel.rotate_to(Orientation.RIGHT)

In [ ]:
laser.trigger()